<a href="https://colab.research.google.com/github/Carl1000000/ALBERT/blob/main/ALBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers sentencepiece pandas scikit-learn torchmetrics

In [3]:
import pandas as pd

# 加载训练集、验证集和测试集
train_df = pd.read_csv("./data/train.tsv", sep="\t", header=0)  # 假设无表头
dev_df = pd.read_csv("./data/dev.tsv", sep="\t", header=0)     # 同上
test_df = pd.read_csv("./data/test.tsv", sep="\t", header=0)                                # 假设CSV有表头

# 检查数据
print("训练集样例:")
print(train_df.head())
print("\n验证集样例:")
print(dev_df.head())
print("\n测试集样例:")
print(test_df.head())

# 检查各数据集形状
print(f"训练集样本数: {len(train_df)}")
print(f"验证集样本数: {len(dev_df)}")
print(f"测试集样本数: {len(test_df)}")

# 检查标签分布
print(train_df["label"].value_counts())
print(dev_df["label"].value_counts())
print(test_df["label"].value_counts())




# 检查缺失值
print(train_df.isnull().sum())
print(dev_df.isnull().sum())
print(test_df.isnull().sum())


训练集样例:
   label                                             text_a
0      1  选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全...
1      1  15.4寸笔记本的键盘确实爽，基本跟台式机差不多了，蛮喜欢数字小键盘，输数字特方便，样子也很...
2      0                               房间太小。其他的都一般。。。。。。。。。
3      0  1.接电源没有几分钟,电源适配器热的不行. 2.摄像头用不起来. 3.机盖的钢琴漆，手不能摸...
4      1  今天才知道这书还有第6卷,真有点郁闷:为什么同一套书有两种版本呢?当当网是不是该跟出版社商量...

验证集样例:
   label                                             text_a
0      1  這間酒店環境和服務態度亦算不錯,但房間空間太小~~不宣容納太大件行李~~且房間格調還可以~~...
1      1  <荐书> 推荐所有喜欢<红楼>的红迷们一定要收藏这本书,要知道当年我听说这本书的时候花很长时...
2      0     商品的不足暂时还没发现，京东的订单处理速度实在.......周二就打包完成，周五才发货...
3      1    ２００１年来福州就住在这里，这次感觉房间就了点，温泉水还是有的．总的来说很满意．早餐简单了些．
4      1  不错的上网本，外形很漂亮，操作系统应该是个很大的 卖点，电池还可以。整体上讲，作为一个上网本...

测试集样例:
   label                                             text_a
0      1                         这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般
1      0  怀着十分激动的心情放映，可是看着看着发现，在放映完毕后，出现一集米老鼠的动画片！开始还怀疑是...
2      0  还稍微重了点，可能是硬盘大的原故，还要再轻半斤就好了。其他要进一步验证。贴的几种膜气泡较多，...
3      1         

In [4]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import torchmetrics as tm

In [5]:

# 数据加载与预处理
class TextDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512, padding="max_length", truncation=True):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.padding = padding
        self.truncation = truncation

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 1]
        label = self.df.iloc[idx, 0]

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding=self.padding,
            truncation=self.truncation,
            return_tensors="pt",
        )

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(label),
        }



# 数据预处理
def preprocess_text(df):
    df["text_a"] = df["text_a"].str.replace(r'<.*?>', '')  # 去除 HTML 标签
    df["text_a"] = df["text_a"].str.replace(r'[^\w\s]', '')  # 去除特殊字符
    return df

train_df = preprocess_text(train_df)
dev_df = preprocess_text(dev_df)
test_df = preprocess_text(test_df)

In [6]:
# 加载预训练模型和tokenizer
model_name = "voidful/albert_chinese_base"

# 加载tokenizer和模型
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/378 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/42.7M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at voidful/albert_chinese_base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# 创建 Dataset 实例
batch_size = 32  # 根据显存调整

train_dataset = TextDataset(train_df, tokenizer)
dev_dataset = TextDataset(dev_df, tokenizer)
test_dataset = TextDataset(test_df, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)


In [8]:

# 定义训练参数
training_args = TrainingArguments(
    output_dir="./results",  # 输出结果保存目录
    num_train_epochs=3,  # 训练轮次
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,  # 热启动步骤
    weight_decay=0.01,  # 正则化
    learning_rate=5e-5,  # 学习率
    # 添加 dataloader 参数
    dataloader_pin_memory=True,
)


# 定义自定义计算指标
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    f1 = f1_score(labels.cpu().numpy(), preds.cpu().numpy())
    accuracy = accuracy_score(labels.cpu().numpy(), preds.cpu().numpy())
    return {"accuracy": accuracy, "f1": f1, "roc_auc": roc_auc_score(labels.cpu().numpy(), torch.nn.functional.softmax(pred.predictions, dim=1).cpu().numpy())[0]}


In [9]:

# 初始化 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # 设置训练集
    eval_dataset=dev_dataset,  # 设置验证集
    compute_metrics=compute_metrics,
)

# 设置数据加载器
trainer.train_dataloader = train_dataloader
trainer.eval_dataloader = dev_dataloader

# 训练模型
trainer.train()



wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: weiyongchao100 (weiyongchao100-jilin-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


Step,Training Loss
500,0.378300


TrainOutput(global_step=858, training_loss=0.29135563101246087, metrics={'train_runtime': 2382.0714, 'train_samples_per_second': 11.519, 'train_steps_per_second': 0.36, 'total_flos': 655716116090880.0, 'train_loss': 0.29135563101246087, 'epoch': 3.0})

In [16]:
# 保存最优模型
best_model = model
best_model.save_pretrained("./results/best_model")


In [31]:
# 加载最优模型
best_model = AutoModelForSequenceClassification.from_pretrained("./results/best_model")


# 初始化新的 Trainer 实例
test_trainer = Trainer(
    model=best_model,
    args=training_args,
    eval_dataset=test_dataset  # 替换为 eval_dataset
)

# 在测试集上进行评估
test_result = test_trainer.evaluate(eval_dataset=test_dataset)
print(test_result)

In [36]:

# 生成预测结果
predictions = test_trainer.predict(test_dataset=test_dataset)
predicted_labels = torch.argmax(torch.tensor(predictions.predictions), dim=1)

# 打印预测结果
print("Predicted Labels:", predicted_labels)

# 保存预测结果
test_df["predicted_label"] = predicted_labels.numpy()
test_df.to_csv("./results/test_predictions.csv", index=False)


Predicted Labels: tensor([0, 0, 0,  ..., 1, 1, 0])


In [42]:
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
# 获取真实标签
true_labels = [sample['labels'] for sample in test_dataset]

# 生成分类报告
report = classification_report(true_labels, predicted_labels, output_dict=True)
print("Classification Report:")
print(report)

# 生成混淆矩阵
cm = confusion_matrix(true_labels, predicted_labels)
print("\nConfusion Matrix:")
print(cm)

# 将分类报告保存为 CSV 文件
df_report = pd.DataFrame(report).transpose()
df_report.to_csv("./results/classification_report.csv", index=True)

# 将混淆矩阵保存为 CSV 文件
df_cm = pd.DataFrame(cm, index=['Class 0', 'Class 1'], columns=['Predicted 0', 'Predicted 1'])
df_cm.to_csv("./results/confusion_matrix.csv", index=True)

Classification Report:
{'0': {'precision': 0.9351081530782029, 'recall': 0.9493243243243243, 'f1-score': 0.942162615255658, 'support': 592.0}, '1': {'precision': 0.9499165275459098, 'recall': 0.9358552631578947, 'f1-score': 0.9428334714167357, 'support': 608.0}, 'accuracy': 0.9425, 'macro avg': {'precision': 0.9425123403120563, 'recall': 0.9425897937411095, 'f1-score': 0.9424980433361969, 'support': 1200.0}, 'weighted avg': {'precision': 0.9426110628085079, 'recall': 0.9425, 'f1-score': 0.9425025157106041, 'support': 1200.0}}

Confusion Matrix:
[[562  30]
 [ 39 569]]
